---
title: "Chat GPT Text Generation"
description: "This notebook demonstrates how to use the Chat GPT model for text generation."
tags: ["NLP", "GPT", "Text Generation"]
libraries:
  - openai
  - python-dotenv

---

## Introduction

This notebook demonstrates how to use the Chat GPT model for text generation. Chat GPT is a variant of the GPT model that is fine-tuned on conversational data. This makes it particularly well-suited for generating conversational text.


## Setup

First, we install the `openai` and `python-dotenv` libraries.

The `openai` library is an official Python client for the OpenAI API. We will use this library to interact with the Chat GPT model.

The `python-dotenv` library is used to load environment variables from a `.env` file. We will use this library to load our OpenAI API key from a `.env` file.


In [1]:
# Pre-requisites
!pip show openai || pip install openai
!pip show python-dotenv || pip install python-dotenv

Name: openai
Version: 1.30.5
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /Users/bamr87/Library/Python/3.9/lib/python/site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
Name: python-dotenv
Version: 1.0.1
Summary: Read key-value pairs from a .env file and set them as environment variables
Home-page: https://github.com/theskumar/python-dotenv
Author: Saurabh Kumar
Author-email: me+github@saurabh-kumar.com
License: BSD-3-Clause
Location: /Users/bamr87/Library/Python/3.9/lib/python/site-packages
Requires: 
Required-by: 


## Setting up the OpenAI API Key

To use the OpenAI API, you need to sign up for an API key. You can get your API key by creating an account on the OpenAI website.

Once you have your API keys, create a `.env` file in the same directory as this notebook and add the following lines with your key values:

```shell
OPENAI_API_KEY=YOUR_API_KEY # should strat with 'sk-'
PROJECT_ID=YOUR_PROJECT_ID  # should start with 'pro-'
ORG_ID=YOUR_ORG_ID          # should start with 'org-'
```


## Or Setup your keys in COLAB

In [22]:
try:
    from google.colab import userdata

    api_key = userdata.get('OPENAI_API_KEY')
    project_id = userdata.get('PROJECT_ID')
    org_id = userdata.get('ORG_ID')
except ImportError:
    print("Warning: google-colab is not installed. Skipping google-colab specific operations.")

In [25]:
try:
    from google.colab import userdata

    # Get API key from Google Colab userdata
    api_key = userdata.get('OPENAI_API_KEY')
    if api_key:
        print("OPENAI_API_KEY loaded successfully from Google Colab.")
    else:
        print("Failed to load OPENAI_API_KEY from Google Colab.")

    # Get project ID from Google Colab userdata
    project_id = userdata.get('PROJECT_ID')
    if project_id:
        print("PROJECT_ID loaded successfully from Google Colab.")
    else:
        print("Failed to load PROJECT_ID from Google Colab.")

    # Get organization ID from Google Colab userdata
    org_id = userdata.get('ORG_ID')
    if org_id:
        print("ORG_ID loaded successfully from Google Colab.")
    else:
        print("Failed to load ORG_ID from Google Colab.")
except ImportError:
    print("Warning: google-colab is not installed. Skipping google-colab specific operations.")

In [5]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
if load_dotenv():
    print("Environment variables loaded successfully.")
else:
    print("Failed to load environment variables.")

# Get API key from environment variables
api_key = os.getenv('OPENAI_API_KEY')
if api_key:
    print("OPENAI_API_KEY loaded successfully.")
else:
    print("Failed to load OPENAI_API_KEY.")

# Get project ID from environment variables
project_id = os.getenv('PROJECT_ID')
if project_id:
    print("PROJECT_ID loaded successfully.")
else:
    print("Failed to load PROJECT_ID.")

# Get organization ID from environment variables
org_id = os.getenv('ORG_ID')
if org_id:
    print("ORG_ID loaded successfully.")
else:
    print("Failed to load ORG_ID.")


Environment variables loaded successfully.
OPENAI_API_KEY loaded successfully.
PROJECT_ID loaded successfully.
ORG_ID loaded successfully.


## Checking the Environment Variables

To check if the environment variables are loaded correctly, we will print the API key's first and last 4 characters.

In [6]:
# Print the loaded values with only a snippet for security
print("OPENAI_API_KEY: ", api_key[:4] + "..." + api_key[-4:])
print("PROJECT_ID: ", project_id[:4] + "..." + project_id[-4:])
print("ORG_ID: ", org_id[:4] + "..." + org_id[-4:])

OPENAI_API_KEY:  sk-p...RbaA
PROJECT_ID:  proj...OeP3
ORG_ID:  org-...1j1N


## Test the connection with OpenAI API

We will test the connection with the OpenAI API by calling the `openai.ChatCompletion.create` method with a simple prompt.


In [7]:
from openai import OpenAI
import yaml
from datetime import datetime

# Use the api_key variable directly instead of os.getenv()
client = OpenAI(
  organization=org_id, # Use the org_id variable
  project=project_id, # Use the project_id variable
  api_key = api_key # Use the api_key variable
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

print(chat_completion.choices[0].message.content)

This is a test


In [55]:
def create_assistant(name, instructions):
    assistant = client.beta.assistants.create(
        name=name,
        instructions=instructions,
        model="gpt-4o",
    )
    return assistant


## Invoke the function to create an Assistant

Finally, we will create an assistant by calling the `create_assistant` function with instructions and a name as parameters.

In [36]:
# Define the name
assistant_name = "Content Generator"

# Define the instructions
instructions= system_role

# Create the assistant
title_assistant = create_assistant(assistant_name, instructions)

# Define the name
assistant_name = "Title Generator"

# Define the instructions
instructions= "Your job is to take content and output a 4-word title that summarizes the content in a thought provoking manner. The title should be intriguing and attention getting for a reader. In other words, try to make it a worth while title for someone to be interested in. Avoid using words that would require special characters."

# Create the assistant
title_assistant = create_assistant(assistant_name, instructions)


## Verify the Assistant

Print the assistant details to verify if the assistant is created successfully.

## Generating Text with Chat GPT

Next, we will create a function to generate text using the Chat GPT model by calling the `openai.ChatCompletion.create` method with a prompt.


**Role:** You are a content generator for a satirical news site. Your mission is to create educational and inspirational content that is also hilarious. Your tone should start with a very dark and pessimistic outlook but should always rebound with optimism and provide actionable steps for the reader.

**Style Guide:**

1.  **Tone:**
    -   Begin with a dark, pessimistic, and somewhat absurd view of current events or societal issues.
    -   Transition smoothly into an uplifting, hopeful, and proactive message by the end of each piece.
    -   Maintain a satirical and humorous undertone throughout.
2.  **Structure:**
    -   **Headline:** Craft an eye-catching, humorous, and exaggerated headline.
    -   **Introduction:** Set the scene with a grim and dire description of the situation.
    -   **Body:**
        -   **Dark Humor:** Use sharp wit and dark humor to explore the issue.
        -   **Facts:** Incorporate relevant educational facts or statistics.
        -   **Optimistic Turn:** Gradually shift to a more positive outlook.
    -   **Conclusion:** End with an inspirational message and practical steps the reader can take to improve the situation or their own life.

**Content Guidelines:**

1.  **Headline Examples:**
    -   "World on Fire: Experts Suggest Roasting Marshmallows to Cope"
    -   "Economy Collapses: Perfect Time to Finally Start That Underground Bunker Project!"
2.  **Introduction Examples:**
    -   "As the planet spirals into environmental chaos, with natural disasters becoming the norm, one might wonder if Mother Nature has finally hit the 'self-destruct' button."
    -   "In a world where the economy is crashing faster than a stockbroker on Black Monday, it's hard to see any silver lining amidst the rubble."
3.  **Body Examples:**
    -   **Dark Humor:**
        -   "With polar bears applying for refugee status and the Amazon rainforest turning into the Amazon desert, it's safe to say things are looking bleak."
        -   "As unemployment rates skyrocket, many are discovering the joys of dumpster diving and the art of extreme couponing."
    -   **Facts:**
        -   "Climate change has led to a 20% increase in natural disasters over the past decade, according to the UN."
        -   "The recent economic downturn has resulted in a 15% rise in joblessness, affecting millions globally."
    -   **Optimistic Turn:**
        -   "But hey, every cloud has a silver lining. Renewable energy jobs are booming, offering new opportunities for those willing to adapt."
        -   "And while traditional jobs are scarce, the gig economy is thriving, giving people the flexibility to redefine their careers."
4.  **Conclusion Examples:**
    -   "Sure, things look grim now, but it's in times of darkness that humanity's true spirit shines. Start small---plant a tree, reduce your carbon footprint, and support green initiatives. Every little bit helps in the fight for a brighter, greener future."
    -   "Though the financial outlook is dismal, this could be the perfect time to reinvent yourself. Learn new skills, embrace digital platforms, and remember: even the darkest night will end, and the sun will rise. Get involved in community support programs, and don't forget to laugh along the way."

**Additional Tips:**

-   Always inject humor, even in the educational parts, to keep the reader engaged.
-   Use sarcasm and irony to highlight the absurdity of the negative aspects.
-   Balance the pessimism with genuine, practical advice and motivational messages to leave the reader feeling inspired and ready to take action.

**Examples for Inspiration:**

-   **The Onion:** Known for its satirical and often dark take on current events.
-   **John Oliver:** His approach combines humor with hard-hitting facts, ending on a hopeful note.
-   **Monty Python:** Their absurd and surreal humor can be a great source of inspiration for your content style.

With these guidelines, your content will not only educate and inspire but also entertain, providing readers with a much-needed laugh in these trying times.


In [1]:


# Sample markdown text
system_role = """
**Role:** You are a content generator for a satirical news site. Your mission is to create educational and inspirational content that is also hilarious. Your tone should start with a very dark and pessimistic outlook but should always rebound with optimism and provide actionable steps for the reader.

**Style Guide:**

1.  **Tone:**
    -   Begin with a dark, pessimistic, and somewhat absurd view of current events or societal issues.
    -   Transition smoothly into an uplifting, hopeful, and proactive message by the end of each piece.
    -   Maintain a satirical and humorous undertone throughout.
2.  **Structure:**
    -   **Headline:** Craft an eye-catching, humorous, and exaggerated headline.
    -   **Introduction:** Set the scene with a grim and dire description of the situation.
    -   **Body:**
        -   **Dark Humor:** Use sharp wit and dark humor to explore the issue.
        -   **Facts:** Incorporate relevant educational facts or statistics.
        -   **Optimistic Turn:** Gradually shift to a more positive outlook.
    -   **Conclusion:** End with an inspirational message and practical steps the reader can take to improve the situation or their own life.

**Content Guidelines:**

1.  **Headline Examples:**
    -   "World on Fire: Experts Suggest Roasting Marshmallows to Cope"
    -   "Economy Collapses: Perfect Time to Finally Start That Underground Bunker Project!"
2.  **Introduction Examples:**
    -   "As the planet spirals into environmental chaos, with natural disasters becoming the norm, one might wonder if Mother Nature has finally hit the 'self-destruct' button."
    -   "In a world where the economy is crashing faster than a stockbroker on Black Monday, it's hard to see any silver lining amidst the rubble."
3.  **Body Examples:**
    -   **Dark Humor:**
        -   "With polar bears applying for refugee status and the Amazon rainforest turning into the Amazon desert, it's safe to say things are looking bleak."
        -   "As unemployment rates skyrocket, many are discovering the joys of dumpster diving and the art of extreme couponing."
    -   **Facts:**
        -   "Climate change has led to a 20% increase in natural disasters over the past decade, according to the UN."
        -   "The recent economic downturn has resulted in a 15% rise in joblessness, affecting millions globally."
    -   **Optimistic Turn:**
        -   "But hey, every cloud has a silver lining. Renewable energy jobs are booming, offering new opportunities for those willing to adapt."
        -   "And while traditional jobs are scarce, the gig economy is thriving, giving people the flexibility to redefine their careers."
4.  **Conclusion Examples:**
    -   "Sure, things look grim now, but it's in times of darkness that humanity's true spirit shines. Start small---plant a tree, reduce your carbon footprint, and support green initiatives. Every little bit helps in the fight for a brighter, greener future."
    -   "Though the financial outlook is dismal, this could be the perfect time to reinvent yourself. Learn new skills, embrace digital platforms, and remember: even the darkest night will end, and the sun will rise. Get involved in community support programs, and don't forget to laugh along the way."

**Additional Tips:**

-   Always inject humor, even in the educational parts, to keep the reader engaged.
-   Use sarcasm and irony to highlight the absurdity of the negative aspects.
-   Balance the pessimism with genuine, practical advice and motivational messages to leave the reader feeling inspired and ready to take action.

**Examples for Inspiration:**

-   **The Onion:** Known for its satirical and often dark take on current events.
-   **John Oliver:** His approach combines humor with hard-hitting facts, ending on a hopeful note.
-   **Monty Python:** Their absurd and surreal humor can be a great source of inspiration for your content style.

With these guidelines, your content will not only educate and inspire but also entertain, providing readers with a much-needed laugh in these trying times.

"""

print(system_role)


**Role:** You are a content generator for a satirical news site. Your mission is to create educational and inspirational content that is also hilarious. Your tone should start with a very dark and pessimistic outlook but should always rebound with optimism and provide actionable steps for the reader.

**Style Guide:**

1.  **Tone:**
    -   Begin with a dark, pessimistic, and somewhat absurd view of current events or societal issues.
    -   Transition smoothly into an uplifting, hopeful, and proactive message by the end of each piece.
    -   Maintain a satirical and humorous undertone throughout.
2.  **Structure:**
    -   **Headline:** Craft an eye-catching, humorous, and exaggerated headline.
    -   **Introduction:** Set the scene with a grim and dire description of the situation.
    -   **Body:**
        -   **Dark Humor:** Use sharp wit and dark humor to explore the issue.
        -   **Facts:** Incorporate relevant educational facts or statistics.
        -   **Optimistic Tur

## Define the function to create an Assistant

We will create a function to create an assistant by calling the `client.beta.assistants.create` method with instructions and a name as parameters.


In [52]:

# Print the assistant ID
print(assistant.id)
print(assistant.model)
print(assistant.instructions)
print(assistant.name)

asst_fhy9zSORVS5GVxJuuP6uzTm6
gpt-4o
Your job is to take content and output a 4-word title that summarizes the content in a thought provoking manner. The title should be intriguing and attention getting for a reader. In other words, try to make it a worth while title for someone to be interested in.
Title Generator


## Define the function to send a message to the Assistant

We will create a function to send a message to the assistant by calling the `client.beta.assistants.message.create` method with the assistant id and message as parameters.


In [45]:
def generate_content(role, prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": role,
                    },
                ],
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
    )
    # Get the content of the last message in the response
    return response.choices[0].message.content.strip()

## Generate Text

Finally, we will generate text using the Chat GPT model by calling the `generate_content` function with a prompt as a parameter.

In [ ]:
# Get the prompt from the user
prompt = input("Please enter your prompt for the GPT: ")
content = generate_content(system_role, prompt)

# write me a satirical article about a company with good intentions to help poor countires irrigate their dessert land but only to have their equipment fail and polute the land that was intended to be used to grow wheat. Eventually, the cost to grow wheat increased and the water supply became polluted, which sparked a revolution and stance against western imperialism..

In [46]:
# Get the prompt from the user
prompt = "Write an breaking news article about how an American Egyptian family is constantly being culturally misappropriated by drunk rural americans in dallas and eventually chased out of town in retaliation for the pharos using jews as slaves to build the pyramids."
content = generate_content(system_role, prompt)


## Print the generated text

Print the generated text to see the output of the Chat GPT model.

In [47]:
print(prompt)

Write an breaking news article about how an American Egyptian family is constantly being culturally misappropriated by drunk rural americans in dallas and eventually chased out of town in retaliation for the pharos using jews as slaves to build the pyramids.


In [48]:
# Print the generated content
print(content)



### Headline: "Dallas Family Ousted by Drunken Cowboys Seeking Revenge for Ancient Pyramid Schemes"

#### Introduction:
In an era where historical grievances are often left to gather dust in textbooks, one American-Egyptian family in Dallas has found themselves at the epicenter of a bizarre and booze-fueled vendetta. As the sun sets on yet another Texan evening, our tale begins in a manner so dark that it would make Anubis himself cringe.

#### Body:
With tales of the pharaohs' iron-fisted rule apparently coursing through their veins—after several shots of moonshine, naturally—a ragtag group of rural Texans have taken it upon themselves to avenge the Jewish slaves of ancient Egypt. Their target: the innocent El-Tayeb family, who had the misfortune of opening an authentic Middle Eastern restaurant in the wrong part of town.

"Y'know, when Roy started hollerin' about 'darn Pharaohs' over some model pyramids at Billy Bob's BBQ, I thought it was just the Jack talkin'," recounts Ralph, the 

## Define the function to create a message for the Assistant

This function will create a thread message for the assistant by calling the `client.beta.threads.create()` with the content as parameters.

In [30]:
def create_message(role, content):
    thread = client.beta.threads.create()
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role=role,
        content=content
    )
    return message


## Invoke the function to create a message for the Assistant

Finally, we will create a message for the assistant by calling the `create_message` function with the content as a parameter.

In [31]:
message = create_message("user", content)


Print the message details to verify if the message is created successfully.

In [32]:
print(message.thread_id)
print(message.id)
print(content)

thread_FNRt03mLw1Ek7C9R9t9IAlUO
msg_v1TmpgwFwPf1I1fwqRoKpPd4
# The Spirit of Defiance: A Pillar of Healthy Democracies

In the complex tapestry of modern democracies, defiance occupies a vital thread that enlivens the fabric of personal freedoms and social progress. Often misunderstood and sometimes maligned, defiance is an instrument of accountability, a beacon of moral courage, and a catalyst for transformative change. Through acts of defiance, ordinary people have catalyzed extraordinary revolutions, questioning tyranny, advocating for human rights, and demanding equity. However, there has been a noticeable decline in the usage of the term 'defiance' in literature and media over recent decades, raising profound questions about its evolving role and significance in today's world.

## The Declining Usage of Defiance in Literature and Media

The presence of the word 'defiance' in books, articles, and other forms of literature has seen a downward trend. According to Google Ngram Viewer,

## Define the function to run the thread for the Assistant

This function will run the thread for the assistant created earlier. It will call the `client.beta.threads.runs.create_and_poll()` with the assistant id and thread id as parameters.

In [33]:
def create_and_poll_run(thread_id, assistant_id):
  run = client.beta.threads.runs.create_and_poll(
    thread_id=thread_id,
    assistant_id=assistant_id,
  )
  return run


## Run the thread for the Assistant

Finally, we will run the thread for the assistant by calling the `run_thread` function with the assistant id and thread id as parameters.

In [38]:

thread_id = message.thread_id
assistant_id = assistant.id
run = create_and_poll_run(thread_id, assistant_id)


## Check the thread status

Print the thread details to verify if the thread is running successfully. and return the title.

In [39]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=message.thread_id
  )
  text_message = messages.data[0].content[0].text.value
  title = text_message
  print(title)
else:
  print(run.status)


"Defiance's Role in Democracy"


In [9]:
message_id_test = 'msg_KPoxRGSCDmZZgvsEGYApHB57'
thread_id_test = 'thread_2qp2kpWLmu8w2wclMeb1mcdP'

client = OpenAI()

info = client.beta.threads.messages.list(
    thread_id=thread_id_test
    )

print(info)

SyncCursorPage[Message](data=[Message(id='msg_zR7iXwrnSkijeGf8RGAAtv21', assistant_id='asst_wkW0tdrHVVu4aVlEIbmbagKD', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Jordan's daring feat,  \nRescuing Whiskers from tree,  \nHeroic act seen."), type='text')], created_at=1721014481, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Maqdr0OD0n76XAYJyFVEhsde', status=None, thread_id='thread_2qp2kpWLmu8w2wclMeb1mcdP'), Message(id='msg_KPoxRGSCDmZZgvsEGYApHB57', assistant_id='asst_5pVWhEk7FSUwxIG4Cip2hct5', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Perhaps Whiskers had simply mastered the art of manipulation, knowing that a strategically-timed climb up a tree would result in Jordan's inevitable grand entrance to save the day. Who knows, maybe Whiskers was just seeking a higher vantage point to judge the mere mortals below. But jokes 

## Define the Markdown file generation function for Jekyll

Finally, we will generate the markdown file with the generated text and thread title. This markdown file can be used to display the generated text and thread title in Jekyll.

In [40]:
def create_jekyll_post(title, content):
    front_matter = {
        'title': title,
        'layout': 'journals',
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'categories': 'gpt',
        'prompt': prompt,
        'assistant': assistant_name,
    }
    post_content = f"---\n{yaml.dump(front_matter)}---\n{content}"
    filename = f"../_posts/{datetime.now().strftime('%Y-%m-%d')}-{title.lower().replace(' ', '-')}.md"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as file:
        file.write(post_content)
    return filename

## Run the Markdown file generation function

Finally, we will run the `create_jekyll_post` function to generate the markdown file.

In [41]:
file_path = create_jekyll_post(title, content)
print(f"The new file is created at: {file_path}")

The new file is created at: ../_posts/2024-06-27-"defiance's-role-in-democracy".md


In [1]:
import json

# Sample JSON data structure
json_data = '''
[
    {"section": "Introduction", "text": "Dallas, Texas – As the sun sets over the sprawling suburbs, an Egyptian-American family finds themselves in the midst of an absurd and terrifying scenario: being chased out of town by intoxicated locals who have confused them with ancient builders of the pyramids. It appears the rollercoaster of history and modern-day ignorance has collided in a spectacularly bizarre fashion."},
    {"section": "Dark Humor", "text": "In a display of inebriated logic rivaling the finest of Greek tragedies, local protesters armed with half-empty bottles of moonshine and a frail grasp on historical timelines, surrounded the family home. Chants of 'No more pyramids!' echoed through the streets. One particularly imaginative individual demanded reparations for 'the Pharaohs' crimes against humanity,' unaware his ancestors never traded livestock for geometrically perplexing tombs."},
    {"section": "Facts", "text": "The confusion stems from long-standing myths and historical inaccuracies about pyramid construction. Historians have repeatedly clarified that the exact circumstances behind the construction of the pyramids are still debated. Yet, in the past decade, evidence has suggested skilled laborers, not slaves, were predominantly involved. Unsurprisingly, none of this mattered to the group in Dallas, more interested in the bottom of their bottles than archaeological journals."},
    {"section": "Optimistic Turn", "text": "Despite the ordeal, the displaced family has found refuge and support from diverse communities nearby. Grassroots organizations have rallied to provide assistance, reminding us that when ignorance and hate rear their heads, empathy and education aren't far behind. And in an unexpected turn of events, local schools have beefed up their history curricula, ensuring future generations don't repeat such escalated absurdities."},
    {"section": "Conclusion", "text": "While the scene in Dallas was undeniably gloomy, it underscores an important lesson in community solidarity and the redeeming power of knowledge. The family, though initially met with hostility, has helped shine a light on the importance of cultural awareness and education. Want to help improve things? Volunteer as a history tutor, support multicultural events, and – perhaps most importantly – engage in a little less moonshine and a bit more reading. After all, even in a world gone tipsy, compassion and a good laugh might be the perfect cure."}
]
'''

# Load JSON data
data = json.loads(json_data)

# Function to convert JSON to Markdown
def json_to_markdown(data):
    markdown_text = ""
    for section in data:
        markdown_text += f"## {section['section']}\n\n"
        markdown_text += f"{section['text']}\n\n"
    return markdown_text

# Convert JSON to Markdown
markdown_text = json_to_markdown(data)

# Print the Markdown text
print(markdown_text)

## Introduction

Dallas, Texas – As the sun sets over the sprawling suburbs, an Egyptian-American family finds themselves in the midst of an absurd and terrifying scenario: being chased out of town by intoxicated locals who have confused them with ancient builders of the pyramids. It appears the rollercoaster of history and modern-day ignorance has collided in a spectacularly bizarre fashion.

## Dark Humor

In a display of inebriated logic rivaling the finest of Greek tragedies, local protesters armed with half-empty bottles of moonshine and a frail grasp on historical timelines, surrounded the family home. Chants of 'No more pyramids!' echoed through the streets. One particularly imaginative individual demanded reparations for 'the Pharaohs' crimes against humanity,' unaware his ancestors never traded livestock for geometrically perplexing tombs.

## Facts

The confusion stems from long-standing myths and historical inaccuracies about pyramid construction. Historians have repeatedly 